In [1]:
# 민원/ cctv/ bus 데이터 주차장 기반 그룹핑하여 개수 세기
# parks_info 로 주차장 데이터 저장 및 랜덤포레스트 돌릴 준비하기
# parks_info 구성(행: 주차장 / 열 : 
#   주소/ 주차장종류/ 운영구분/ 총주차면/ 평일유료/ 토요일유료/ 공휴일유료/ 평일시작/ 평일종료/ 토요일시작/ 토요일종료/ 공휴일시작/ 공휴일종료/ 
#   기본주차요금/ 기본주차시간/ 추가단위요금/ 추가단위시간/ 경도/ 위도/ 평일운영시간/ 토요일운영시간/ 공휴일운영시간/ 1시간요금
#   r300민원/ r300cctv/ r300bus

#pip install geopy

from geopy.distance import great_circle
import pandas as pd
import folium

/Users/minss/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [11]:
class CountByPark:
    
    def __init__(self, df, lat, lon, dist):
        """_summary_

        Args:
            df (_type_): 데이터프레임
            lat (_type_): 중심 위도(주차장 위도)
            lon (_type_): 중심 경도(주차장 경도)
            dist (_type_): 기준 거리(km)
        """
        
        self.df = df
        self.lat = lat
        self.lon = lon
        self.dist = dist
    
    def filter_by_rectangle(self):
        """
        사각범위 내 데이터 필터링
        : 반경범위 하기 앞서 데이터 양 줄이기,
        """
        lat_min = self.lat - 0.01*self.dist
        lat_max = self.lat + 0.01*self.dist
        
        lon_min = self.lon - 0.01*self.dist
        lon_max = self.lon + 0.01*self.dist
        
        self.points = [[lat_min, lon_min],[lat_max, lon_max]]
        
        result = self.df.loc[
            (self.df['lat']>lat_min) &
            (self.df['lat']<lat_max) &
            (self.df['lon']>lon_min) &
            (self.df['lon']<lon_max)
        ]
        result.index = range(len(result))
        
        return result
    
    def filter_by_radius(self):
        """
        반경 범위 내 데이터 필터링
        """
        
        # 사각범위 내 데이터 필터링
        tmp = self.filter_by_rectangle()
        
        # 기준 좌표 포인트
        center = (self.lat, self.lon)
        
        result = pd.DataFrame()
        for index, row in tmp.iterrows():
            # 개별 좌표 포인트
            point = (row['lat'], row['lon'])
            d = great_circle(center, point).kilometers
            if d <= self.dist:
                result = pd.concat( [result, tmp.iloc[index, :].to_frame().T] )
                
            
        result.index = range(len(result))
        
        return result
    
    def plot_by_radius(self, df):
        """
        반경 범위 내 데이터 플로팅
        """
        
        m = folium.Map(location = [self.lat, self.lon], zoom_start = 14)
        
        for idx, row in df.iterrows():
            lat_ = row['lat']
            lon_ = row['lon']
            
            folium.Marker(location = [lat_, lon_],
                          radius = 15,
                          tooltip = row['주차장코드']).add_to(m)
            
        folium.Circle(radius = dist*1000,
                      location = [lat,lon],
                      color = "#ff7800",
                      fill_color = "#ffff00",
                      fill_opacity = 0.2
                      ).add_to(m)
        
        return m

In [10]:
parks_df = pd.read_csv("src/parks_mod.csv")
cctv_df = pd.read_csv("cctv.csv")
complaints_df = pd.read_csv("reports.csv")
bus_df = pd.read_csv("bus.csv", engine= 'python')

In [12]:
complaints_df.shape

(3064538, 6)

In [13]:
parks_df = parks_df.rename(columns = {"경도": "lon", "위도" : 'lat'})
cctv_df = cctv_df.rename(columns = {"경도": "lon", "위도" : 'lat'})
complaints_df = complaints_df.rename(columns = {"경도": "lon", "위도" : 'lat'})
bus_df = bus_df.rename(columns = {"경도": "lon", "위도" : 'lat'})

In [14]:
complaints_df['요일'].unique()

array(['Weekday', 'Saturday', 'Holiday'], dtype=object)

In [15]:
complaints_df.head

<bound method NDFrame.head of               민원접수일    민원접수시간                       주소         lon        lat  \
0        2021-09-29  19:29:00      서울특별시 강서구 강서로15길 49  126.843247  37.532089   
1        2021-09-29  18:48:00        성북구 오패산로19길 34-5   127.033761  37.609537   
2        2021-09-29  18:47:00  장위로21다길 59-19 주소지 앞도로 외  127.045741  37.616406   
3        2021-09-29  18:47:00     서울특별시 강북구 오패산로30길 13  127.034685  37.613820   
4        2021-09-29  18:46:00    서울특별시 강서구 강서로18길 52-5  126.848703  37.534293   
...             ...       ...                      ...         ...        ...   
3064533  2024-03-13  12:33:00        서울특별시 마포구 동교로 144  126.918137  37.555325   
3064534  2024-03-13  12:34:00    서울특별시 서초구 강남대로101안길 4  127.018191  37.516667   
3064535  2024-03-13  12:34:00   서울특별시 마포구 동교로12길 41-16  126.913712  37.552550   
3064536  2024-03-13  12:28:00          서울 강동구 성안로 11-5  127.128981  37.525097   
3064537  2024-03-13  23:57:00         (신림동) 남부순환로 1637  126.932592  37.484883  

In [16]:
#bus 위경도 str -> float

def str2float(x):
    return float(x)

#bus_df['lat'] = bus_df['lat'].apply(str2float)
#bus_df['lon'] = bus_df['lon'].apply(str2float)

bus_df['lat'] = bus_df['lat'].apply(lambda x: float(x.replace("\ufeff","")))
bus_df['lon'] = bus_df['lon'].apply(lambda x: float(x.replace("\ufeff","")))


#bus_df['lat'][1]
#bus_df['lon'][1]

In [9]:
parks_df['lat'][1]

37.48149637668632

In [ ]:
'''
test


lat = parks_df['lat'][13]
lon = parks_df['lon'][13]
dist = 0.3

cbp_cctv = CountByPark(cctv_df, lat, lon, dist)
cctv_rectangle = cbp_cctv.filter_by_rectangle()
cctv_radius = cbp_cctv.filter_by_radius()

print(f"""
      {"="*50}
      중심위도 : {cbp_cctv.lat}
      중심경도 : {cbp_cctv.lon}
      기준거리 : {cbp_cctv.dist} km
      사각 범위 내 데이터 필터링 결과 : {len(cctv_rectangle):,}건
      반경 범위 내 데이터 필터링 결과 : {len(cctv_radius):,}건
      {"="*50}
      """)

#df
#parks_df.loc[13,'cctv_r300'] = 5
parks_df.loc[[13]]
cctv_radius
'''


      중심위도 : 37.50574874860143
      중심경도 : 127.05534382983517
      기준거리 : 0.3 km
      사각 범위 내 데이터 필터링 결과 : 6건
      반경 범위 내 데이터 필터링 결과 : 5건
      


,고정형CCTV지번주소,lat,lon
0,삼성동 142-46,37.50589,127.05269
1,삼성동 143-35,37.50631,127.05277
2,대치동 902,37.50425,127.05624
3,대치동 893,37.5048,127.05595
4,강남구 삼성동 143-49,37.506675,127.054666


In [17]:
parks_df.shape

# complaints_r300/ cctv_r300/ bus_r300 열 추가
# r300: 반경 300m
parks_df['cctv_r300'] =''
parks_df['bus_r300'] =''
parks_df['complaints_r300'] =''

In [18]:
# cctv 그룹핑
dist = 0.3

for i in range(parks_df.shape[0]):
      lat = parks_df['lat'][i]
      lon = parks_df['lon'][i]
      
      #count cctv by park
      cbp_cctv = CountByPark(cctv_df, lat, lon, dist)
      cctv_rectangle = cbp_cctv.filter_by_rectangle()
      cctv_radius = cbp_cctv.filter_by_radius()
      parks_df.loc[i,'cctv_r300'] = len(cctv_radius)
      
      #count bus by park
      
      #count complaints by park


parks_df.loc[1:3]

,주차장코드,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,...,추가단위시간,lon,lat,평일운영시간,토요일운영시간,공휴일운영시간,1시간 요금,cctv_r300,bus_r300,complaints_r300
1,1040225,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,0,24,...,5.0,127.048218,37.481496,24,24,24,2400.0,0,,
2,1033754,강남구 개포동 1273-0,NW,1,194.0,Y,N,N,0,24,...,5.0,127.052365,37.475646,24,24,24,2400.0,3,,
3,173472,강남구 개포동 13-2,NW,1,168.0,Y,N,N,0,24,...,5.0,127.079307,37.494938,24,24,24,1200.0,2,,


In [19]:
# bus 그룹핑

dist = 0.3

for i in range(parks_df.shape[0]):
      lat = parks_df['lat'][i]
      lon = parks_df['lon'][i]
      
      #count cctv by park
      cbp_bus = CountByPark(bus_df, lat, lon, dist)
      bus_rectangle = cbp_bus.filter_by_rectangle()
      bus_radius = cbp_bus.filter_by_radius()
      parks_df.loc[i,'bus_r300'] = len(bus_radius)
      
      #count bus by park
      
      #count complaints by park


parks_df.loc[1:3]

,주차장코드,주소,주차장종류,운영구분,총주차면,평일유료,토요일유료,공휴일유료,평일시작,평일종료,...,추가단위시간,lon,lat,평일운영시간,토요일운영시간,공휴일운영시간,1시간 요금,cctv_r300,bus_r300,complaints_r300
1,1040225,강남구 개포동 1266-0,NW,1,97.0,Y,N,N,0,24,...,5.0,127.048218,37.481496,24,24,24,2400.0,0,1,
2,1033754,강남구 개포동 1273-0,NW,1,194.0,Y,N,N,0,24,...,5.0,127.052365,37.475646,24,24,24,2400.0,3,5,
3,173472,강남구 개포동 13-2,NW,1,168.0,Y,N,N,0,24,...,5.0,127.079307,37.494938,24,24,24,1200.0,2,4,


In [20]:
#sampled_complaints_df = complaints_df.sample(frac=1/3, random_state=42).reset_index(drop=True)

#sampled_complaints_df.shape

In [21]:
# 민원 그룹핑

from tqdm import tqdm

dist = 0.3

def checkType(complaints):
      if complaints['요일'] == 'Weekday' : return '평일'
      elif complaints['요일'] == 'Saturday' : return '토요일'
      else : return '공휴일'
      
#간추려진 민원 df
def checkTime(complaints_df, time_df, daytype):
      
    drop_index = []
    
    for j in range(complaints_df.shape[0]):
        start = time_df[daytype + '시작']
        end = time_df[daytype + '종료']
        t = complaints_df.loc[j, '민원접수시간'].split(':')
        mod_time = int(t[0]) * 10000 + int(t[1]) * 100 + int(t[2])
        if not (int(start) * 10000 <= mod_time <= int(end) * 10000):
            drop_index.append(j)
            
    return complaints_df.drop(index=drop_index)

daytype_mapping = {
    '평일': 'Weekday',
    '토요일': 'Saturday',
    '공휴일': 'Holiday'
}

                  
def process_complaints_by_day(parks_df, complaints_df, daytype, save_path):
    parks_df = parks_df.copy()
    parks_df[f'complaints_r300_{daytype}'] = 0
    dist = 0.3
    filtered_complaints = complaints_df[complaints_df['요일'] == daytype_mapping[daytype]]

    for i in tqdm(range(parks_df.shape[0]), desc=f"Processing {daytype}"):
        lat = parks_df.loc[i, 'lat']
        lon = parks_df.loc[i, 'lon']
        cbp = CountByPark(filtered_complaints, lat, lon, dist)
        radius_df = cbp.filter_by_radius()
        time_df = parks_df.loc[i, [f'{daytype}시작', f'{daytype}종료']].to_frame().T
        result_df = checkTime(radius_df, time_df.iloc[0], daytype)
        parks_df.loc[i, f'complaints_r300_{daytype}'] = len(result_df)

    parks_df.to_csv(save_path, index=False)              
      
# 주차장 기준 민원 그룹핑      
def process_complaints_by_day(parks_df, complaints_df, daytype, save_path):
    parks_df = parks_df.copy()
    parks_df[f'complaints_r300_{daytype}'] = 0
    dist = 0.3
    filtered_complaints = complaints_df[complaints_df['요일'] == daytype_mapping[daytype]]

    for i in tqdm(range(parks_df.shape[0]), desc=f"Processing {daytype}"):
        lat = parks_df.loc[i, 'lat']
        lon = parks_df.loc[i, 'lon']
        cbp = CountByPark(filtered_complaints, lat, lon, dist)
        radius_df = cbp.filter_by_radius()
        time_df = parks_df.loc[i, [f'{daytype}시작', f'{daytype}종료']].to_frame().T
        result_df = checkTime(radius_df, time_df.iloc[0], daytype)
        parks_df.loc[i, f'complaints_r300_{daytype}'] = len(result_df)

    parks_df.to_csv(save_path, index=False)
      
#complaints_radius
#result_compalaints_df

#parks_df.loc[1:3]

In [ ]:
parks_df = pd.read_csv("src/parks_mod.csv")
complaints_df = pd.read_csv("reports.csv")

parks_df = parks_df.rename(columns={"경도": "lon", "위도": "lat"})
complaints_df = complaints_df.rename(columns={"경도": "lon", "위도": "lat"})

print(complaints_df['요일'].value_counts().reindex(['Weekday', 'Saturday', 'Holiday']))

요일
Weekday     2027023
Saturday     490527
Holiday      546988
Name: count, dtype: int64


"\nprocess_complaints_by_day(parks_df, complaints_df, '평일', 'src/parks_weekday.csv')\nprocess_complaints_by_day(parks_df, complaints_df, '토요일', 'src/parks_saturday.csv')\nprocess_complaints_by_day(parks_df, complaints_df, '공휴일', 'src/parks_holiday.csv')\n"

In [ ]:
process_complaints_by_day(parks_df, complaints_df, '평일', 'src/parks_weekday.csv')
process_complaints_by_day(parks_df, complaints_df, '토요일', 'src/parks_saturday.csv')
process_complaints_by_day(parks_df, complaints_df, '공휴일', 'src/parks_holiday.csv')

In [ ]:
parks_df.to_csv("src/parks_addGroupedInfo.csv", index = False)

In [ ]:
#complaints_radius.loc[0]


민원접수일              2023-05-15
민원접수시간               21:24:00
주소        개포동 1256//7304 도로침범
lon                127.047215
lat                 37.479681
요일                    Weekday
Name: 0, dtype: object

In [ ]:
#type = '평일'
#parks_df.loc[1,[type+'시작','평일종료','토요일시작','토요일종료','공휴일시작','공휴일종료']]

평일시작      0
평일종료     24
토요일시작     0
토요일종료    24
공휴일시작     0
공휴일종료    24
Name: 1, dtype: object